<a href="https://colab.research.google.com/github/PragateeD/Gen-AI-PROJECT/blob/main/chatbot_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install Dependencies
!pip install -q langchain-community langchain-core faiss-cpu gradio pandas sentence-transformers cohere
!pip install --upgrade langchain cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 M

In [4]:
# 2. Import Libraries
import pandas as pd
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
import gradio as gr


In [5]:
# 3. Load & Clean Data (Minimal)
# Load raw CSV
df = pd.read_csv("/content/flipkart_laptop.csv")

# Basic cleaning
df.columns = df.columns.str.strip().str.replace(" ", "_")
df = df.dropna(subset=["Description"])

# Fix encoding issues in price
df['Prices'] = df['Prices'].astype(str).str.replace(r'[^\d]', '', regex=True).astype(int)


In [6]:
# 4. Create LangChain Documents
documents = [
    Document(
        page_content=row['Description'],
        metadata={"name": row["Product_name"], "price": row["Prices"]}
    )
    for _, row in df.iterrows()
]


In [7]:
# 5. Create FAISS Vectorstore
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


<ipython-input-7-9abcd5c65e83>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# 6. Setup LLM Chain (Cohere)
cohere_api_key = "VhKGyVtrr840QnyCPlDupNO05vA6JSKCCTBxj4ac"  # Replace this
llm = Cohere(cohere_api_key=cohere_api_key)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


<ipython-input-8-0cdeafb5c224>:3: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere(cohere_api_key=cohere_api_key)


In [10]:
# 7. Define Filtering Logic
def extract_price_limit(query):
    match = re.search(r'under\s*₹?\s*(\d+)', query)
    return int(match.group(1)) if match else None

def respond(query):
    price_limit = extract_price_limit(query)
    result = qa_chain({"query": query})
    answer = result["result"]
    sources = result["source_documents"]

    matching = []
    for doc in sources:
        meta = doc.metadata
        if price_limit is None or int(meta["price"]) <= price_limit:
            matching.append(f"- {meta['name']} (₹{meta['price']})")

    if not matching:
        matching.append("No exact match under that budget, but here are similar options.")
        matching += [f"- {doc.metadata['name']} (₹{doc.metadata['price']})" for doc in sources]

    return f"{answer}\n\nMatching Products:\n" + "\n".join(matching)


In [11]:
# 8. Create Gradio UI
gr.Interface(
    fn=respond,
    inputs=gr.Textbox(placeholder="Ask about laptops (e.g., under ₹50000)..."),
    outputs="text",
    title="💻 Flipkart Laptop Recommender",
    examples=[
        "Best laptops under ₹30000",
        "Suggest laptop for gaming under 40000",
        "Give laptops with Intel i5 processor"
    ]
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab470c2b7530cf25f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
# Install dependencies
!pip install -q langchain-community langchain-core faiss-cpu gradio pandas sentence-transformers cohere

# ---------------------- Step 1: Imports ----------------------
import pandas as pd
import re
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain.schema import Document
import gradio as gr

# ---------------------- Step 2: Load and Clean Data ----------------------
df = pd.read_csv("/content/flipkart_laptop.csv")

# Minimal but essential cleaning
df = df.dropna(subset=['Description', 'Product_name', 'Prices'])
df['Prices'] = df['Prices'].astype(str).str.extract(r'(\d[\d,]*)')[0].str.replace(',', '').astype(int)
df.reset_index(drop=True, inplace=True)

# ---------------------- Step 3: Create Embedding Documents ----------------------
def create_docs(df):
    docs = []
    for _, row in df.iterrows():
        text = f"{row['Product_name']} | {row['Description']} | ₹{row['Prices']}"
        docs.append(Document(
            page_content=text,
            metadata={
                "name": row["Product_name"],
                "price": row["Prices"]
            }
        ))
    return docs

documents = create_docs(df)

# ---------------------- Step 4: Build Vectorstore ----------------------
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

# ---------------------- Step 5: Setup Retrieval QA Chain ----------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=Cohere(cohere_api_key="VhKGyVtrr840QnyCPlDupNO05vA6JSKCCTBxj4ac"),  # Replace with your real key
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True
)

# ---------------------- Step 6: Smart Post-Retrieval Filtering ----------------------
def extract_filters(query):
    price_limit = None
    brand = None

    # More comprehensive price extraction
    price_matches = re.findall(r"₹?\s*(\d+)\s*(?:to|-|and)\s*₹?\s*(\d+)", query, re.IGNORECASE)
    if price_matches:
        price_limit = int(price_matches[0][1])  # Take upper limit
    else:
        price_match = re.search(r"(?:under|below|less than)\s*₹?\s*(\d+)", query, re.IGNORECASE)
        if price_match:
            price_limit = int(price_match.group(1))

    brands = ['acer', 'hp', 'dell', 'lenovo', 'asus', 'msi', 'apple', 'samsung']
    for b in brands:
        if re.search(rf"\b{b}\b", query, re.IGNORECASE):
            brand = b.lower()
            break

    return price_limit, brand
def respond(query):
    price_limit, brand = extract_filters(query)

    all_docs = vectorstore.similarity_search(query, k=20)

    matches = []
    seen_products = set()

    for doc in all_docs:
        name = doc.metadata["name"]
        price = doc.metadata["price"]

        if price_limit is not None and price > price_limit:
            continue
        if brand is not None and brand.lower() not in name.lower():
            continue

        product_id = f"{name}_{price}"
        if product_id in seen_products:
            continue
        seen_products.add(product_id)

        matches.append({
            "name": name,
            "price": price,
            "doc": doc
        })

    matches.sort(key=lambda x: x["price"])

    if not matches:
        return f"No {brand.capitalize() if brand else ''} laptops found under ₹{price_limit or 'your budget'}. Try adjusting your requirements."

    # Dynamic heading
    heading = "Here are"
    if brand:
        heading += f" {brand.capitalize()}"
    heading += " laptops"
    if price_limit:
        heading += f" under ₹{price_limit}"
    heading += ":\n\n"

    response = heading
    for match in matches:
        response += f"- {match['name']} (₹{match['price']})\n"

    return response

# ---------------------- Step 7: Gradio UI ----------------------
gr.Interface(
    fn=respond,
    inputs=gr.Textbox(placeholder="Ask about laptops..."),
    outputs="text",
    title="Laptop Recommender",
    examples=[
        "Best laptops under ₹30000",
        "Suggest laptop for gaming under 40000",
        "Give Acer laptops with i5 processor"
    ]
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aab7c150b71b7025c0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
